# 2020-02-24 超分辨率重建
## 概述
超分辨率重建技术（Super-Resolution）是指从观测到的低分辨率图像重建出相应的高分辨率图像。
SR可分为两类:
1. 从多张低分辨率图像重建出高分辨率图像
2. 从单张低分辨率图像重建出高分辨率图像。

基于深度学习的SR，主要是基于单张低分辨率的重建方法，即Single Image Super-Resolution (SISR)

## SR based on DL
1. EDSR：可实现4倍放大，精度最高， 速度较慢：https://github.com/Saafke/EDSR_Tensorflow
2. ESPCN：可实现4倍放大，速度最快， 可实现实时视频处理：https://github.com/fannymonori/TF-ESPCN
3. FSRCNN： 可实现3倍放大，速度快， 可实时视频处理：https://github.com/Saafke/FSRCNN_Tensorflow
4. LapSRN： 可实现8倍放大：https://github.com/fannymonori/TF-LAPSRN

## 批量对文件夹中的图像进行重建
crop

In [1]:
import argparse
import time
import cv2
from cv2 import dnn_superres
import os
from utils import crop


In [15]:
def super_res_image(img_path, out_path, model_path):
    modelName = model_path.split("/")[-1].split("_")[0].lower()
    modelScale = model_path.split("_x")[-1]
    modelScale = int(modelScale[:modelScale.find(".")])
    sr = dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)
    sr.setModel(modelName, modelScale)
    _, filename = os.path.split(img_path)

    if os.path.splitext(filename)[-1] not in (".png", ".jpg", ".jpeg", ".bmp"):
        print(f"can't open file: {filename}")
    else:
        orig = cv2.imread(img_path, 1)
        if orig.shape[0] <= 2400:
            
            upscaled = sr.upsample(orig)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        else:
            # crop, sr then merge
            crop_path, num_cols, num_rows = crop.clip_one_picture(path=os.path.split(img_path)[0]+"/",filename=os.path.split(img_path)[-1],cols=1200,rows=1200)
            
            for i, filename in enumerate(os.listdir(crop_path)):
                img_path = crop_path + "/" + filename
                croped = cv2.imread(img_path, 1)
                upscaled = sr.upsample(croped)
                cv2.imwrite(img_path, upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
                
            merge_upscaled = crop.merge_picture(crop_path,num_cols,num_rows)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), merge_upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

def super_res(file_path, out_path, model_path):

    if os.path.isfile(file_path):
        super_res_image(img_path=file_path, out_path=out_path, model_path=model_path)
    elif os.path.isdir(file_path):
        for filename in os.listdir(file_path):
            img_path = file_path + "/" + filename
            super_res_image(img_path=img_path, out_path=out_path, model_path=model_path)
    else:
        print(f"cannot read path: {file_path}, please try again.")

    return




选用FSRCNN模型，复原home/span/super_resolution/examples/examples/中的图像，保存在./result中

In [16]:
def main():
    model_path = "./models/FSRCNN_x3.pb"
    file_path = "./examples/examples/" 
    img_path = file_path + "child.png"
    out_path = file_path + "result"
    super_res(file_path, out_path, model_path)
    print("success!!!")

main()

can't open file: .DS_Store
can't open file: result
can't open file: child.png_croped_1200x1200
裁剪所得3列图片，2行图片.
裁剪所得图片的存放地址为：./examples/examples//child.png_croped_1200x1200/
./examples/examples//child.png_croped_1200x1200/ ['./examples/examples//child.png_croped_1200x1200/child_0_1.png', './examples/examples//child.png_croped_1200x1200/child_0_0.png', './examples/examples//child.png_croped_1200x1200/child_1_2.png', './examples/examples//child.png_croped_1200x1200/child_1_1.png', './examples/examples//child.png_croped_1200x1200/child_0_2.png', './examples/examples//child.png_croped_1200x1200/child_1_0.png']
success!!!


In [18]:
!rm -r ./examples/examples/result/

In [19]:
! mkdir ./examples/examples/result

## 计算评价函数
1. ssim：结构化相似度，衡量图片相似度的指标，通过亮度(luminance)、对比度(contrast)和结构(structure)三个方面估计感知结构信息的变化。范围: -1～1
2. psnr：峰值信噪比，即峰值信号的能量与噪声的平均能量之比。PSNR指标越高，说明图像质量越好。

In [ ]:
import math
import numpy as np
# from PIL import Image
from scipy.signal import convolve2d
"""
PSNR
"""
def mse(img1, img2):
    mse = np.mean( (img1/255. - img2/255.) ** 2)
    return mse

def psnr(img1, img2):
    mse = np.mean((img1/1.0 - img2/1.0) ** 2 )
    if mse < 1.0e-10:
        return 100
    return 10 * math.log10(255.0**2/mse)

"""
SSIM
"""
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
#     2D gaussian mask - should give the same result as MATLAB's
#     fspecial('gaussian',[shape],[sigma])

    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h
 
def filter2(x, kernel, mode='same'):
    return convolve2d(x, np.rot90(kernel, 2), mode=mode)
 
def compute_ssim(im1, im2, k1=0.01, k2=0.03, win_size=11, L=255):
    if not im1.shape == im2.shape:
        raise ValueError("Input Imagees must have the same dimensions")
    if len(im1.shape) > 2:
        raise ValueError("Please input the images with 1 channel")

    M, N = im1.shape
    C1 = (k1*L)**2
    C2 = (k2*L)**2
    window = matlab_style_gauss2D(shape=(win_size,win_size), sigma=1.5)
    window = window/np.sum(np.sum(window))

    if im1.dtype == np.uint8:
        im1 = np.double(im1)
    if im2.dtype == np.uint8:
        im2 = np.double(im2)

    mu1 = filter2(im1, window, 'valid')
    mu2 = filter2(im2, window, 'valid')
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = filter2(im1*im1, window, 'valid') - mu1_sq
    sigma2_sq = filter2(im2*im2, window, 'valid') - mu2_sq
    sigmal2 = filter2(im1*im2, window, 'valid') - mu1_mu2

    ssim_map = ((2*mu1_mu2+C1) * (2*sigmal2+C2)) / ((mu1_sq+mu2_sq+C1) * (sigma1_sq+sigma2_sq+C2))

    return np.mean(np.mean(ssim_map))

